# ACE Optimisation using XPOT

This is a simple example of optimising parameters for ACE models using the XPOT package. We start by initialising an instance of the `PACE` class, which reads in the input file and sets up everything required for the PACE model to be fitted by the optimiser. The input file can be found in the examples folder.

In [1]:
from xpot.models import PACE

mlip = PACE("ace_input.hjson")

Now that the ML potential object is set up, we can inspect the parameters that have been defined for optimisation in the input file, and check that those have been correctly parsed by the `PACE` class. The second cell prints the entire parameter dictionary, and will be used throughout the optimisation and updated with the new parameter values for each iteration.

In [2]:
print(mlip.optimisation_space)

{(('potential', 'embeddings', 'ALL', 'fs_parameters', '3'),): Real(low=0.1, high=2, prior='uniform', transform='identity'), (('potential', 'bonds', 'ALL', 'radparameters', '0'),): Real(low=1, high=10, prior='uniform', transform='identity'), (('potential', 'bonds', 'ALL', 'rcut'),): Real(low=3, high=6, prior='uniform', transform='identity'), (('potential', 'bonds', 'ALL', 'dcut'),): Real(low=0.001, high=0.1, prior='uniform', transform='identity'), (('potential', 'functions', 'number_of_functions_per_element'),): Integer(low=10, high=100, prior='uniform', transform='identity')}


In [3]:
print(mlip.ace_total)

{'cutoff': 6, 'seed': 42, 'metadata': OrderedDict([('purpose', 'Potential fit')]), 'data': OrderedDict([('filename', '/home/inputs/train_data_fixed.pkl.gzip'), ('test_filename', '/home/inputs/test_data_fixed.pkl.gzip')]), 'potential': OrderedDict([('deltaSplineBins', 0.001), ('elements', ['Si']), ('embeddings', OrderedDict([('ALL', OrderedDict([('npot', 'FinnisSinclairShiftedScaled'), ('fs_parameters', [1, 1, 1, 'skopt.space.Real(0.1, 2)']), ('ndensity', 2), ('rho_core_cut', 3000), ('drho_core_cut', 500)]))])), ('rankmax', ''), ('bonds', OrderedDict([('ALL', OrderedDict([('radbase', 'SBessel'), ('radparameters', ['skopt.space.Real(1,10)']), ('rcut', 'skopt.space.Real(3,6)'), ('dcut', 'skopt.space.Real(0.001, 0.1)'), ('r_in', ''), ('delta_in', ''), ('core-repulsion', ''), ('NameofCutoffFunction', 'cos')]))])), ('functions', OrderedDict([('number_of_functions_per_element', 'skopt.space.Integer(10,100)'), ('ALL', OrderedDict([('nradmax_by_orders', [15, 6, 4, 3, 2, 2]), ('lmax_by_orders', 

Now that the ML potential object is set up, we can import and initialise the optimiser which wraps around the ML potential object. the kwargs dictionary passes optimization parameters to the optimizer, possible values and their defaults are documented at [`scikit_optimize.Optimizer`](https://scikit-optimize.github.io/dev/modules/generated/skopt.Optimizer.html#skopt.Optimizer).

In [4]:
from xpot.optimiser import NamedOptimiser

kwargs = {
        "n_initial_points": 5,
        }

opt = NamedOptimiser(mlip.optimisation_space, mlip.sweep_path, kwargs)

Initialised CSV Files


We limit the number of iterations to 10, and perform 10 fits and record their loss values. By using `optimiser.run_optimisation` we run a fit and evaluate all of the parameters & results produced from a single iteration.

In [5]:
n_calls = 10

while opt.iter <= n_calls:
    opt.run_optimisation(mlip.fit, path = mlip.sweep_path)

...

Finally, after we have completed the optimisation across the desired number of iterations (or achieved the desired loss value), we produce the final tables of the errors and parameters from the optimisation sweep.

In [ ]:
opt.tabulate_final_results(mlip.sweep_path)